In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
cd /content/drive/MyDrive/Colab/캐글놀이터

In [ ]:
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import numpy as np

# 얼굴인식 AI 라이브러리
import dlib
from imutils import face_utils

# 영상 출력용
from IPython.display import HTML
from base64 import b64encode

DATA_PATH = "./data"
HEIGTH = int(2160 * 0.2)
WIDTH =int(3840 * 0.2)


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [ ]:
# 동영상 불러서 HTML 코드로 colab에 동영상 재생
mp4 = open(DATA_PATH + '/video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode() 
HTML(""" <video width=500 controls> <source src="%s" type="video/mp4"> </video> """ % data_url)


In [ ]:
#사이즈가 너무 커서 디텍딩에 오랜 시간걸림
#따라서 먼저 동영상 전체 리사이즈 먼저 해서 저장
cap = cv2.VideoCapture(DATA_PATH + "/video.mp4")
ret, img = cap.read()

fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('./output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (WIDTH, HEIGTH))

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    try:
        img = cv2.resize(img,(WIDTH, HEIGTH))
        #opencv에서 불러들때는 BGR로 불러오기 때문에 색이 이상하게 된다
        #save할때는 색공간이 제대로 들어감....
        #그래서 save할땐 아래는 실행 할 필요 없음
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        out.write(img)
        plt.imshow(img)
        
    except e:
        print(e)

out.release()
cap.release()

In [ ]:
#리사이즈 한 동영상을 불러와서 디텍팅
cap = cv2.VideoCapture("./output.mp4")
original_img = ""           # 오리지널 이미지
rectangle_img = ""          # 크롭하고 68개의 점이 있는 이미지
no_circle_img = ""
shape = ""                   # 점의 위치가 저장되어있는 리스트
left, top, right, bottom = "", "", "", ""
while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    dats = detector(img)
    if len(dats) > 0:
        dat = dats[0]
        left, top, right, bottom = dat.left(), dat.top(), dat.right(), dat.bottom()
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        shape = predictor(img, dat)
        shape = face_utils.shape_to_np(shape)

        # python의 2차원 배열 인덱싱은 행(세로), 열(가로) 순서
        # 따라서 원하는 이미지 위치를 자르려면 [up: bottom, left:right]로 잘라야함
        rectangle_img = img[top:bottom, left:right].copy()
        no_circle_img = rectangle_img.copy()
        for (x,y) in shape:
            # 크롭한 이미지에서 점을 찾아야 하기 때문에 원래의 좌표에서 제일 왼쪽, 제일 위쪽 값 빼주기
            cv2.circle(rectangle_img, (int(x - left), int(y - top)), 2, (2555, 0, 0),  -1)

    original_img = img
    plt.imshow(rectangle_img)


In [ ]:
left_img = no_circle_img.copy()
right_img = no_circle_img.copy()
mouse_img = no_circle_img.copy()

# 좌우 눈, 입 좌표 저장
left_eye = shape[36:42]
right_eye = shape[42:48]
mouse = shape[48:68]

# 하나씩 테스트
for (x,y) in left_eye:
    cv2.circle(left_img, (int(x - left), int(y - top)), 2, (2555, 0, 0),  -1)
for (x,y) in right_eye:
    cv2.circle(right_img, (int(x - left), int(y - top)), 2, (2555, 0, 0),  -1)
for (x,y) in mouse:
    cv2.circle(mouse_img, (int(x - left), int(y - top)), 2, (2555, 0, 0),  -1)

plt.subplot(131)
plt.imshow(left_img)
plt.subplot(132)
plt.imshow(right_img)
plt.subplot(133)
plt.imshow(mouse_img)

In [ ]:
print(left_eye)
print(left_eye[2:][0])